## Prerequisites - Environment Setup

Before starting this workshop, you need to set up a Python development environment. Follow these steps:

### Step 1: Create Virtual Environment
From the **root folder** of this repository, run:
```bash
python -m venv .venv
```

### Step 2: Activate Virtual Environment
- **Windows**: `.venv\Scripts\activate`
- **macOS/Linux**: `source .venv/bin/activate`

### Step 3: Create .env Configuration File
Create a `.env` file in this directory (or the parent directory) with your Azure OpenAI credentials:
```bash
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini
PROJECT_ENDPOINT=https://your-resource.services.ai.azure.com/api/projects/your-project
PEOPLE_AGENT_ID=asst-your-people-agent-id
KNOWLEDGE_AGENT_ID=asst-your-knowledge-agent-id
```

**Important:** Never commit your `.env` file to version control! Add it to `.gitignore`.

### Step 4: Select Virtual Environment in VS Code
1. Click on the **kernel selector** in the top-right corner of this notebook
2. Select **"Select Another Kernel..."**
3. Choose **"Python Environments..."**
4. Select the `.venv` environment you just created

---

# Python Semantic Kernel Agents Workshop

This notebook demonstrates modern AI agent development using Microsoft Semantic Kernel in Python with Azure OpenAI services.

## Workshop Focus Areas
1. **Generic Agent** - Build basic conversational agents using Semantic Kernel
2. **Azure AI Foundry Agents** - Demonstrate specialized agents like PeopleLookupAgent and KnowledgeFinderAgent
3. **Group Chat System** - Multi-agent collaboration with built-in orchestration

## Key Technologies
- **Semantic Kernel Python** - Microsoft's AI orchestration framework for Python
- **Azure OpenAI** - GPT-4 models for intelligent responses
- **Azure AI Foundry** - Cloud-based specialized AI agents
- **Agent Coordination** - Multi-agent systems for complex tasks

Let's build sophisticated AI agent systems using Python and Semantic Kernel!

### Step 1: Install Required Packages

In [ ]:
# Install required packages for Semantic Kernel agents
import subprocess
import sys

def install_requirements():
    """Install required packages for the workshop."""
    print("Installing Semantic Kernel and required packages...")
    
    packages = [
        "semantic-kernel>=1.3.0",
        "azure-ai-projects",
        "azure-identity",
        "python-dotenv",
        "aiohttp",
        "pyyaml"
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ Installed {package}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package}: {e}")
    
    print("✓ Package installation completed!")
    print("Ready to import modules and create agents!")

# Run the installation
install_requirements()

In [ ]:
# Import required modules for Semantic Kernel agents
import os
import asyncio
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("✓ Environment variables loaded from .env file")
print("✓ All modules imported successfully!")
print("Ready to configure agents and start building!")

In [ ]:
# Verify environment configuration
required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY", 
    "AZURE_OPENAI_DEPLOYMENT_NAME"
]

print("Checking required environment variables...")
missing_vars = []

for var in required_vars:
    value = os.getenv(var)
    if value:
        print(f"✓ {var}: {value[:20]}...")
    else:
        missing_vars.append(var)
        print(f"❌ {var}: Not set")

if missing_vars:
    print(f"\n❌ Missing required variables: {', '.join(missing_vars)}")
    print("Please check your .env file configuration.")
else:
    print("\n✓ All required environment variables are configured!")
    print("Ready to create agents!")

In [ ]:
# Create a Generic Agent using Semantic Kernel
import sys
import os

# Add the current directory to Python path to import our agent classes
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'python', 'sk'))

# Import the agent classes
from agents.semantic_kernel_agents import SemanticKernelGenericAgent
from shared import AgentConfig, AgentType

async def create_generic_agent():
    """Create and initialize a generic Semantic Kernel agent."""
    
    # Create agent configuration
    config = AgentConfig(
        name="GenericAssistant",
        agent_type=AgentType.GENERIC,
        model_config={
            "endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
            "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
            "deployment_name": os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
        },
        instructions="""You are a helpful AI assistant that can answer questions and have conversations.
        
        Your capabilities include:
        - Answering general questions
        - Helping with problem-solving
        - Providing explanations and information
        - Having natural conversations
        
        Always be helpful, accurate, and professional in your responses."""
    )
    
    # Create and initialize the agent
    agent = SemanticKernelGenericAgent(config)
    await agent.initialize()
    
    print("✓ Generic Semantic Kernel Agent created successfully!")
    print(f"Agent Name: {agent.name}")
    print(f"Agent Type: {agent.config.agent_type}")
    print("Ready to test the agent!")
    
    return agent

# Create the generic agent
generic_agent = await create_generic_agent()

### Test the Generic Agent

# Test the Generic Agent
async def test_generic_agent():
    """Test the generic agent with a sample question."""
    
    # Test message - you can modify this to ask different questions
    user_message = "Hello! What is artificial intelligence and how does it work?"
    
    print(f"User: {user_message}")
    print("Sending message to agent...")
    
    try:
        # Get response from the agent
        response = await generic_agent.process_message(user_message)
        
        print(f"\nAgent ({generic_agent.name}): {response.content}")
        print(f"\nResponse received successfully!")
        print("✓ Generic agent test completed!")
        print("You can modify the user_message above to test different questions.")
        
    except Exception as e:
        print(f"❌ Error testing agent: {e}")
        print("Please check your Azure OpenAI configuration.")

# Run the test
await test_generic_agent()

In [ ]:
### Create Azure AI Foundry Specialized Agents

In [ ]:
# Create Azure AI Foundry Specialized Agents
from agents.semantic_kernel_agents import SemanticKernelAzureFoundryAgent

async def create_foundry_agents():
    """Create Azure AI Foundry specialized agents."""
    
    print("Creating Azure AI Foundry specialized agents...")
    
    # People Lookup Agent
    people_config = AgentConfig(
        name="PeopleLookupAgent",
        agent_type=AgentType.AZURE_FOUNDRY,
        agent_id=os.getenv("PEOPLE_AGENT_ID"),
        project_endpoint=os.getenv("PROJECT_ENDPOINT"),
        instructions="""You are a People Lookup Agent, an expert at finding and providing information about people, contacts, and team members.

Your expertise includes:
• Directory Searches: Finding people by name, role, department, or skills
• Team Coordination: Understanding organizational structure and relationships  
• Contact Discovery: Providing appropriate contact information
• Role Identification: Matching people to specific tasks or expertise needs

Guidelines for responses:
- Be helpful and accurate when providing people information
- If you don't know a specific person, explain what information would be helpful to find them
- Suggest ways to locate people or get contact information
- Always try to be helpful for any people-related query"""
    )
    
    # Knowledge Finder Agent
    knowledge_config = AgentConfig(
        name="KnowledgeFinderAgent",
        agent_type=AgentType.AZURE_FOUNDRY,
        agent_id=os.getenv("KNOWLEDGE_AGENT_ID"),
        project_endpoint=os.getenv("PROJECT_ENDPOINT"),
        instructions="""You are a Knowledge Finder Agent, a specialist at searching, retrieving, and organizing information from various knowledge sources.

Your expertise includes:
• Document Search: Finding relevant documents, wikis, and knowledge bases
• Research Assistance: Discovering and synthesizing information from multiple sources
• Information Organization: Structuring and summarizing complex knowledge
• Context Provision: Connecting related information and concepts

Guidelines for responses:
- Focus on finding the most relevant and accurate information
- Provide clear summaries with key points highlighted
- Help users understand complex information through clear explanations
- Structure information in a logical, easy-to-digest format"""
    )
    
    # Create and initialize the agents
    people_agent = SemanticKernelAzureFoundryAgent(people_config)
    knowledge_agent = SemanticKernelAzureFoundryAgent(knowledge_config)
    
    await people_agent.initialize()
    await knowledge_agent.initialize()
    
    print("✓ Azure AI Foundry agents created successfully!")
    print(f"1. {people_agent.name} - Specializes in people and contact information")
    print(f"2. {knowledge_agent.name} - Expert at finding and organizing knowledge")
    print("These agents demonstrate specialized capabilities from Azure AI Foundry!")
    
    return people_agent, knowledge_agent

# Create the foundry agents
people_agent, knowledge_agent = await create_foundry_agents()

In [ ]:
# Test Azure AI Foundry Agents
async def test_foundry_agents():
    """Test each specialized agent with questions in their area of expertise."""
    
    print("Testing Azure AI Foundry specialized agents...")
    print()
    
    # Test cases for each agent
    test_cases = [
        {
            "agent": people_agent,
            "question": "I need to find someone who can help with Python development. Who should I contact?"
        },
        {
            "agent": knowledge_agent, 
            "question": "What are the best practices for implementing microservices architecture?"
        }
    ]
    
    for i, test_case in enumerate(test_cases, 1):
        agent = test_case["agent"]
        question = test_case["question"]
        
        try:
            print(f"Test {i}: {agent.name}")
            print(f"Question: {question}")
            print("-" * 50)
            
            # Get response from the agent
            response = await agent.process_message(question)
            print(f"Response: {response.content}")
            
            print("=" * 70)
            print()
            
        except Exception as e:
            print(f"❌ Error testing {agent.name}: {e}")
            print("=" * 70)
            print()
    
    print("✓ Specialized agent testing completed!")
    print("Each agent demonstrated expertise in their specialized domain.")

# Run the foundry agent tests
await test_foundry_agents()

In [ ]:
### Create Group Chat with All Three Agents

# Create Group Chat with Multiple Agents
from routers.semantic_kernel_router import HybridSemanticKernelRouter
from shared import SessionManager, AgentRegistry

async def create_group_chat():
    """Create a group chat system with all three agents."""
    
    print("Creating Agent Group Chat system...")
    
    # Create session manager and agent registry
    session_manager = SessionManager()
    agent_registry = AgentRegistry()
    
    # Register all our agents
    agent_registry.register_agent(generic_agent)
    agent_registry.register_agent(people_agent)
    agent_registry.register_agent(knowledge_agent)
    
    # Create the router for orchestrating group chat
    router = HybridSemanticKernelRouter(
        agent_registry=agent_registry,
        session_manager=session_manager
    )
    
    print("✓ Agent Group Chat created successfully!")
    print(f"Participants: {', '.join([agent.name for agent in [generic_agent, people_agent, knowledge_agent]])}")
    print("Ready for multi-agent collaboration!")
    
    return router

# Create the group chat system
group_chat_router = await create_group_chat()

In [ ]:
# Test Group Chat with Complex Question
async def test_group_chat():
    """Test the group chat system with a complex question that benefits from multiple perspectives."""
    
    # Complex question that requires multiple agent expertise
    complex_question = """We're launching a new AI-powered project management tool. 
We need to understand:
1. Who should be involved in the development team?
2. What technical knowledge and best practices should we consider?
3. How can we ensure the project is successful?

Please provide a comprehensive response with insights from different perspectives."""
    
    print("Testing Group Chat with Complex Question:")
    print(f"Question: {complex_question}")
    print("=" * 80)
    print()
    
    try:
        # Create a new session for this group chat
        session_id = "group_chat_test"
        
        # Send the question to the group chat router
        print("Starting group chat collaboration...")
        response = await group_chat_router.route_message(
            message=complex_question,
            session_id=session_id,
            user_id="user_test"
        )
        
        print(f"Group Chat Response:")
        print(f"{response.content}")
        print()
        print("✓ Group Chat completed successfully!")
        print("The agents worked together to provide a comprehensive response!")
        
    except Exception as e:
        print(f"❌ Error in group chat: {e}")
        print("Please check your Azure OpenAI and Azure AI Foundry configuration.")

# Run the group chat test
await test_group_chat()

### Try Your Own Group Chat Question

In [ ]:
# Try Your Own Group Chat Question
async def custom_group_chat_test():
    """Test the group chat with your own custom question."""
    
    print("Custom Group Chat Test")
    print("Feel free to modify the question below to explore different topics!")
    print()
    
    # MODIFY THIS QUESTION to test different scenarios:
    custom_question = """How can we improve remote team collaboration? 
Consider finding the right people, knowledge management, and general best practices."""
    
    print(f"Your Question: {custom_question}")
    print("=" * 60)
    
    try:
        # Create a session for the custom test
        session_id = "custom_group_chat"
        
        print("Starting custom group chat...")
        response = await group_chat_router.route_message(
            message=custom_question,
            session_id=session_id,
            user_id="custom_user"
        )
        
        print(f"Group Response:")
        print(f"{response.content}")
        print()
        print("✓ Custom group chat completed!")
        print("Feel free to modify the custom_question above to explore different topics!")
        
    except Exception as e:
        print(f"❌ Error in custom group chat: {e}")

# Run the custom test
await custom_group_chat_test()

## Integration with Python API Backend

This workshop connects with the actual Python Semantic Kernel API backend. The agents you've created here follow the same patterns used in the production API.

### Key Integration Points:
1. **Agent Architecture** - Uses the same agent classes as the backend API
2. **Group Chat System** - Implements the HybridSemanticKernelRouter for multi-agent coordination  
3. **Azure Integration** - Same Azure OpenAI and Azure AI Foundry setup as the API
4. **Session Management** - Compatible session and agent registry patterns

### Production Features Available:
- **REST API Endpoints** - `/group-chat` for multi-agent conversations
- **Agent Registry** - Dynamic agent management and selection
- **Session Management** - Persistent conversation history
- **Error Handling** - Robust error handling and logging

### Try the API:
You can test the running API backend at `http://localhost:8001` (if running) with endpoints like:
- `GET /health` - Check system health
- `GET /agents` - List available agents  
- `POST /group-chat` - Start multi-agent conversations

The notebook agents you've created demonstrate the core capabilities that power the full API system!